In [ ]:
from database_handler import *

In [ ]:
session = create_connection()

In [2]:
from transformation_handler import *

In [3]:
mydata = clean_all_data(1000000)

c:\Users\olasm\anaconda3\envs\etl_env\lib\site-packages\pandas\core\arrays\timedeltas.py:908: RuntimeWarning: invalid value encountered in cast
  base = data.astype(np.int64)
c:\Users\olasm\anaconda3\envs\etl_env\lib\site-packages\pandas\core\arrays\timedeltas.py:912: RuntimeWarning: invalid value encountered in cast
  data = (base * m + (frac * m).astype(np.int64)).view("timedelta64[ns]")


In [5]:
import datetime
sonoma = f"https://data.sonomacounty.ca.gov/resource/924a-vesw.json?$select=id,type,breed,color,sex,date_of_birth,intake_date,outcome_date,intake_type,outcome_type"

In [19]:
print(datetime.datetime.now())

2023-10-12 19:07:37.802788


In [6]:
print(datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f"))
#no need for this

2023-10-12T21:16:25.500501


In [8]:
response = requests.get(sonoma, params={'$limit': 1000000,'$where': "intake_date >= '2023-10-08' or outcome_date >= '2023-10-08'",'$order': 'intake_date ASC'})
df = pd.DataFrame(response.json())

In [2]:
from prehook import execute_prehook

In [3]:
execute_prehook()

Connecting to database "animal_intakes_outcomes"...
Connection was successful.
Closing connection to database...
Connection is closed.
